# Scenarios Generation Main

In [ ]:
#Name of the datafile
datafile = "usecase_initialdata_fr.xlsx"

In [ ]:
import scenarios_generation 
import TraitementDataconflit
import pandas as pd
import pickle
import tools
import analyse_bdd_general
import time

In [ ]:
#Datafile treatment
a = TraitementDataconflit.tablecreation(datafile)

data_decision = a[3]
data_opinions = a[0]
data_positions = a[1]
data_user = a[2]
data_initialstate = a[6]
data_logicalconstraint = a[4]
data_pplconstraint = a[5]
data_goals = a[7]

## System creation

In [ ]:
my_system = scenarios_generation.System({},[],{})

D = scenarios_generation.D_formation(data_decision)

ens_decisions_acteurs_objets = scenarios_generation.decisions_acteurs_objets(D,data_decision)

my_system = scenarios_generation.variables_formation(data_opinions, my_system)
   
my_system = scenarios_generation.acteurs_formation(data_user, ens_decisions_acteurs_objets,my_system,data_decision,data_positions,data_opinions)         
    
my_systeme = scenarios_generation.pp_formations(my_system,data_positions)

for i in data_initialstate.index:
    my_system.v.get(i).valeur_initiale(data_initialstate.loc[i].Valeur)
     
etat_mon_systeme = scenarios_generation.EtatSysteme(data_positions, data_opinions, my_system.renvoyer_etats_variables())

print("System creation done")

## Scenario Tree Generation

Chose the maximum tree depth and the type of generation

In [ ]:
MAX_PROF = 3

#if input == [] : all scenarios are generated
#if input == [x] : return 1 scenario for x scenarios
#if input == n : return the first n scenarios
cutcriteria = []

In [ ]:
a = time.time()
dict_etat = {} #the file with the system states and there numbers
dict_decision = {}
indice = 1
nbscenarios = [0]
dict_etat.update({1:etat_mon_systeme})
root = scenarios_generation.Node(False,1,[],{}, 0,None)
compteur = 0

with open('stockage_scenariotest.pkl','wb')as writer :
    t3 = scenarios_generation.treegeneration3(writer,cutcriteria,nbscenarios, indice, dict_etat,root,root,my_system, data_opinions, 
                            data_positions, data_pplconstraint,D, data_logicalconstraint, MAX_PROF, data_goals)

print("End generation")
b=time.time()
print(b-a)

## Storage and Scenarios saving

In [ ]:
dict_etat1 = {}
for i in dict_etat :
    dict_etat1.update({i:dict_etat.get(i).etat_variables})
dict_etatex = pd.DataFrame(dict_etat1)
dict_etatex.to_excel('dict_states.xlsx')

data = []
with open('stockage_scenariotest.pkl','rb')as file :
    while True :
        try:
            data.append(pickle.load(file))
        except EOFError:
            break


print("data done")

In [ ]:
tools.storage(data,'stockage_scenarios.pkl')
tools.storage(dict_etat, 'stockagedict_etat.pkl')
tools.storage(D, 'stockage_d.pkl')
tools.storage(my_system,'stockage_systeme.pkl')

print("Storage done")

tools.allscenariostoreadbetter('allscenarios_output.txt', data, dict_etat)
print(analyse_bdd_general.statistiques(data, dict_etat, data_opinions))